<a href="https://colab.research.google.com/github/Ayy4s/datamanipulationpart1/blob/main/sunum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=c97ab04223a9862aab6ed4e4afeb544d6bcba09bc2e64f747953775db563a4ae
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
sc = SparkSession.builder.master("local[*]").appName("data6").getOrCreate()


In [ ]:
csv_file_path = "/content/drive/MyDrive/ford_gobike_data.csv"

df = sc.read.option("header", "true").csv(csv_file_path)

In [ ]:
sc

In [ ]:
df.show()

+----------+-----------------+----------------+--------+---------------+--------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+-------------------+-------------------+----------------------+------------------+
|start_time|start_time_second|start_time_am_pm|end_time|end_time_second|end_time_am_pm|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|          distance|     start_datetime|       end_datetime|time_difference_second|               fee|
+----------+-----------------+----------------+--------+---------------+--------------+----------------+--------------------+----------------------+--------------------

In [ ]:
grouped_df = df.groupBy("user_type").agg(sum("time_difference_second"))
grouped_df.show()

+----------+---------------------------+
| user_type|sum(time_difference_second)|
+----------+---------------------------+
|Subscriber|               2.88830647E8|
|  Customer|               2.82547043E8|
+----------+---------------------------+



In [ ]:
mean_value = df.select(mean("time_difference_second")).collect()[0][0]
mean_value

1099.4375408889744

20 min

In [ ]:
total_count = df.agg(count("fee")).collect()[0][0]
total_count

519700

In [ ]:
total_count_20 = df.filter(df["time_difference_second"] > 1200)
total_count_20.agg(count("fee")).collect()[0][0]

82380

In [ ]:
total_count_20 = df.filter(df["time_difference_second"] < 1200)
total_count_20.agg(count("fee")).collect()[0][0]

437187

 Identify the most popular routes by combining start and end station information and suggest potential improvements to the bike network based on these findings
Seasonal usage patterns: Analyze the usage patterns across different seasons (spring, summer, fall, winter). This could help to determine if there are certain seasons where usage is significantly higher or lower.

In [ ]:
df_with_new_column = df.withColumn("route", concat_ws(" - ", col("start_station_name"), col("end_station_name")))
df_with_new_column.show()

+----------+-----------------+----------------+--------+---------------+--------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+-------------------+-------------------+----------------------+------------------+--------------------+
|start_time|start_time_second|start_time_am_pm|end_time|end_time_second|end_time_am_pm|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|          distance|     start_datetime|       end_datetime|time_difference_second|               fee|               route|
+----------+-----------------+----------------+--------+---------------+--------------+----------------+--------------------+-

In [ ]:
route_grouped = df_with_new_column.groupBy("route").agg(count("fee").alias("total"))
route_grouped.show()

+--------------------+-----+
|               route|total|
+--------------------+-----+
|20th St at Bryant...|   55|
|Commercial St at ...|   39|
|47th St at San Pa...|    1|
|Division St at Po...|  143|
|18th St at Noe St...|   38|
|Spear St at Folso...|   11|
|2nd Ave at E 18th...|   68|
|29th St at Tiffan...|   43|
|Golden Gate Ave a...|    3|
|5th St at Brannan...|  197|
|Valencia St at 21...|   53|
|Lakeshore Ave at ...|  104|
|Golden Gate Ave a...|   68|
|13th St at Frankl...|    1|
|College Ave at Ha...|   26|
|Union Square (Pow...|   23|
|Fruitvale BART St...|    1|
|Franklin St at 9t...|   44|
|San Francisco Cit...|   10|
|Powell St BART St...|   25|
+--------------------+-----+
only showing top 20 rows



In [ ]:
route_sorted = route_grouped.orderBy(col("total").desc())
route_sorted.show()

+--------------------+-----+
|               route|total|
+--------------------+-----+
|San Francisco Fer...| 3344|
|The Embarcadero a...| 1793|
|The Embarcadero a...| 1734|
|Berry St at 4th S...| 1686|
|2nd St at S Park ...| 1509|
|Montgomery St BAR...| 1485|
|San Francisco Fer...| 1401|
|Steuart St at Mar...| 1361|
|2nd St at Townsen...| 1274|
|19th Street BART ...| 1196|
|The Embarcadero a...| 1190|
|Howard St at Beal...| 1183|
|Market St at 10th...| 1148|
|San Francisco Fer...| 1105|
|Howard St at 2nd ...| 1100|
|Bay Pl at Vernon ...| 1054|
|Grand Ave at Perk...| 1041|
|The Embarcadero a...| 1037|
|5th St at Howard ...| 1031|
|Montgomery St BAR...|  987|
+--------------------+-----+
only showing top 20 rows



In [ ]:
route_sorted.write.csv("route_output.csv", header=True, mode="overwrite")

In [ ]:
data_df = df_with_new_column.withColumn("Month", month(col("start_datetime")))
data_df.show()

+----------+-----------------+----------------+--------+---------------+--------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+-------------------+-------------------+----------------------+------------------+--------------------+-----+
|start_time|start_time_second|start_time_am_pm|end_time|end_time_second|end_time_am_pm|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|          distance|     start_datetime|       end_datetime|time_difference_second|               fee|               route|Month|
+----------+-----------------+----------------+--------+---------------+--------------+----------------+----------

In [ ]:
from pyspark.sql.functions import month, when

trip_data = df_with_new_column.withColumn(
    "season",
    when((month("start_datetime") >= 12) | (month("start_datetime") <= 2), "Winter")
    .when((month("start_datetime") >= 3) & (month("start_datetime") <= 5), "Spring")
    .when((month("start_datetime") >= 6) & (month("start_datetime") <= 8), "Summer")
    .when((month("start_datetime") >= 9) & (month("start_datetime") <= 11), "Fall")
    .otherwise("Unknown")
)
trip_data.show()


+----------+-----------------+----------------+--------+---------------+--------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+-------------------+-------------------+----------------------+------------------+--------------------+------+
|start_time|start_time_second|start_time_am_pm|end_time|end_time_second|end_time_am_pm|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|          distance|     start_datetime|       end_datetime|time_difference_second|               fee|               route|season|
+----------+-----------------+----------------+--------+---------------+--------------+----------------+--------

In [ ]:
gb_data = trip_data.groupBy("season").agg(count("fee"))
gb_data.show()

+------+----------+
|season|count(fee)|
+------+----------+
|  Fall|    303107|
|Winter|     86539|
|Summer|    130054|
+------+----------+



Identify "dead-zones": Determine if there are certain end stations where bikes tend to accumulate and from where they are rarely rented again. These could be areas where users frequently end their trips but rarely start new ones.

In [ ]:
start_gb = df.groupBy("start_station_id").agg(count("fee").alias("start_total"))
end_gb = df.groupBy("end_station_id").agg(count("fee").alias("end_total"))

In [ ]:
merged_df = start_gb.join(end_gb, start_gb["start_station_id"] == end_gb["end_station_id"], "outer")
merged_df.show()

+----------------+-----------+--------------+---------+
|start_station_id|start_total|end_station_id|end_total|
+----------------+-----------+--------------+---------+
|              10|       2310|            10|     2696|
|             100|       1567|           100|     1541|
|             101|        873|           101|      929|
|             102|       1071|           102|     1375|
|             105|       1481|           105|      889|
|             106|       1100|           106|      976|
|             107|       1607|           107|     1355|
|             108|        925|           108|      991|
|             109|       3053|           109|     3214|
|              11|       3474|            11|     3893|
|             110|       1021|           110|     1123|
|             112|       2147|           112|     2783|
|             113|       1133|           113|      889|
|             114|       2516|           114|     2676|
|             115|       1624|           115|   

In [ ]:
merged_df = merged_df.withColumn("ratio", (col("end_total") / col("start_total")))
merged_df.show()

+----------------+-----------+--------------+---------+------------------+
|start_station_id|start_total|end_station_id|end_total|             ratio|
+----------------+-----------+--------------+---------+------------------+
|              10|       2310|            10|     2696|1.1670995670995672|
|             100|       1567|           100|     1541|0.9834077855775367|
|             101|        873|           101|      929|1.0641466208476518|
|             102|       1071|           102|     1375|1.2838468720821663|
|             105|       1481|           105|      889| 0.600270087778528|
|             106|       1100|           106|      976|0.8872727272727273|
|             107|       1607|           107|     1355|0.8431860609831985|
|             108|        925|           108|      991|1.0713513513513513|
|             109|       3053|           109|     3214|1.0527350147396004|
|              11|       3474|            11|     3893|1.1206102475532527|
|             110|       

In [ ]:
ordered_df = merged_df.orderBy(col("ratio").desc())
ordered_df.show()

+----------------+-----------+--------------+---------+------------------+
|start_station_id|start_total|end_station_id|end_total|             ratio|
+----------------+-----------+--------------+---------+------------------+
|             268|          1|           268|        2|               2.0|
|             157|        617|           157|     1021|1.6547811993517019|
|             151|        149|           151|      234| 1.570469798657718|
|             247|        739|           247|     1118|1.5128552097428958|
|             339|          2|           339|        3|               1.5|
|             226|         41|           226|       61|1.4878048780487805|
|             245|       1667|           245|     2342|1.4049190161967606|
|             263|         35|           263|       49|               1.4|
|              30|      12546|            30|    17378|1.3851426749561613|
|             131|         47|           131|       63|1.3404255319148937|
|             259|       